In [114]:
import os

In [115]:
pwd

'D:\\Ineuron\\Deep_Learning\\Chicken_Disease_Classification'

In [116]:
os.chdir('D:\Ineuron\Deep_Learning\Chicken_Disease_Classification')

In [117]:
pwd

'D:\\Ineuron\\Deep_Learning\\Chicken_Disease_Classification'

In [118]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen = True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [1]:
from src.constants import *
from src.utils.common import read_yaml, create_directories

ModuleNotFoundError: No module named 'src'

In [120]:
os.getcwd()

'D:\\Ineuron\\Deep_Learning\\Chicken_Disease_Classification'

In [121]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        create_directories([self.config.artifacts_root])

    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [122]:
import os
import urllib.request as request
import zipfile
from src import logger
from src.utils.common import get_size

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  

    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [123]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-08-10 18:23:01,308: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-08-10 18:23:01,311: INFO: common: yaml file: params.yaml loaded successfully]
[2023-08-10 18:23:01,313: INFO: common: created directory at: artifacts]
[2023-08-10 18:23:01,347: INFO: common: created directory at: artifacts/data_ingestion]
[2023-08-10 18:23:04,209: INFO: 939663607: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 11616915
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "f8e60d9910a8d7109af1808d267fc959e3e676ffa8337f031f2ea2613b7b4618"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 0CD4:218BCC:14A840:1CB5FC:64D4DDAB
Accept-Ranges: bytes
Date: Thu, 10 Aug 2023 12:53:02 GMT
Via: 1.1 varnish
X-Served-By: cache-hyd1100023-HYD
X-Cache: